In [2]:
import geemap
import ee
import os

In [3]:
email = "mt-hvi-zonalstats-service-acco@mt-hvi-zonalstats.iam.gserviceaccount.com"
key_file = "/Users/natebender/Desktop/repo/mt-hvi-zonalstats-6da42ca28c80.json"

# Authenticate and initialize
credentials = ee.ServiceAccountCredentials(email=email, key_file=key_file)
ee.Initialize(credentials)

In [15]:
# Define the area of interest (aoi) as the boundary of Montana.
aoi = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("STUSPS", "MT"))
tracts = ee.FeatureCollection("TIGER/2020/TRACT").filter(ee.Filter.eq("STATEFP", "30"))
usa = ee.FeatureCollection("TIGER/2018/States") \
            .filter(ee.Filter.neq('STATEFP', '02'))  # Exclude Alaska (STATEFP '02')

# Load SRTM terrain data and clip to the USA boundaries.
#terrain = ee.Image('USGS/SRTMGL1_003').clip(usa) # full USA terrain view
terrain = ee.Image('USGS/SRTMGL1_003').clip(aoi) # just Montana
terrain_ft = terrain.multiply(3.28084).rename('elevation_ft') # convert meters to ft


In [16]:
# dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2021_REL/TCC/v2021-4')\
#     .filterBounds(aoi) \
#     .filter(ee.Filter.calendarRange(2021, 2021, 'year'))

# nlcd_tree_canopy = dataset.select('NLCD_Percent_Tree_Canopy_Cover').first().clip(aoi.geometry())

In [38]:
census_tract_geoid = "30063001801"
aoi = ee.FeatureCollection("TIGER/2020/TRACT")\
         .filter(ee.Filter.eq("GEOID", census_tract_geoid))

dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2021_REL/TCC/v2021-4')\
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(2021, 2021, 'year'))

nlcd_tree_canopy = dataset.select('NLCD_Percent_Tree_Canopy_Cover').first().clipToCollection(aoi)


In [39]:
tree_canopy_legend_dict = {
    '0-10%': '#ffffff',
    '10-20%': '#d9f0d3',
    '20-30%': '#addd8e',
    '30-40%': '#78c679',
    '40-50%': '#41ab5d',
    '50-60%': '#238443',
    '60-70%': '#006837',
    '70-80%': '#004529',
    '80-90%': '#003300',
    '90-100%': '#002200'
}

tree_canopy_vis_params = {
    'min': 0,
    'max': 100,
    'palette': ['ffffff', '006400']
}

m = geemap.Map()
m.set_center(-110, 47, 6)

m.add_layer(
    nlcd_tree_canopy,
    tree_canopy_vis_params,
    'Tree Canopy Cover'
)

m.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
m.add_legend(title="Tree Canopy Cover (%)", legend_dict=tree_canopy_legend_dict)
m

Map(center=[47, -110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [41]:
geoids = [
    "30063000100", "30063000203", "30063000204", "30063000205", "30063000206",
    "30063000300", "30063000400", "30063000501", "30063000502", "30063000700",
    "30063000801", "30063000802", "30063000901", "30063000902", "30063001001",
    "30063001002", "30063001100", "30063001200", "30063001302", "30063001303",
    "30063001304", "30063001401", "30063001402", "30063001501", "30063001502",
    "30063001601", "30063001602", "30063001801", "30063001802"
]

In [42]:
def export_nlcd_tree_canopy(geoid):
    # Define AOI for the specific census tract
    aoi = ee.FeatureCollection("TIGER/2020/TRACT")\
             .filter(ee.Filter.eq("GEOID", geoid))
    
    # Load NLCD Tree Canopy dataset for 2021 and clip to the AOI
    dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2021_REL/TCC/v2021-4')\
        .filterBounds(aoi) \
        .filter(ee.Filter.calendarRange(2021, 2021, 'year'))

    nlcd_tree_canopy = dataset.select('NLCD_Percent_Tree_Canopy_Cover').first().clipToCollection(aoi)
    
    # Define the export path with the GEOID in the file name
    canopy_path = os.path.join(output_dir, f'canopy_export_{geoid}.tif')
    
    # Export the image
    geemap.ee_export_image(
        nlcd_tree_canopy, 
        filename=canopy_path, 
        scale=30,  # Original 30m resolution
        region=aoi.geometry(), 
        file_per_band=False
    )
    print(f"Exported GEOID: {geoid}")

In [43]:
for geoid in geoids:
    export_nlcd_tree_canopy(geoid)

print("All export tasks have been submitted.")

Generating URL ...
Please wait ...
Data downloaded to /Users/natebender/Desktop/repo/mslacounty_hvi/outputs/canopy_export_30063000100.tif
Exported GEOID: 30063000100
Generating URL ...
Please wait ...
Data downloaded to /Users/natebender/Desktop/repo/mslacounty_hvi/outputs/canopy_export_30063000203.tif
Exported GEOID: 30063000203
Generating URL ...
Please wait ...
Data downloaded to /Users/natebender/Desktop/repo/mslacounty_hvi/outputs/canopy_export_30063000204.tif
Exported GEOID: 30063000204
Generating URL ...
Please wait ...
Data downloaded to /Users/natebender/Desktop/repo/mslacounty_hvi/outputs/canopy_export_30063000205.tif
Exported GEOID: 30063000205
Generating URL ...
Please wait ...
Data downloaded to /Users/natebender/Desktop/repo/mslacounty_hvi/outputs/canopy_export_30063000206.tif
Exported GEOID: 30063000206
Generating URL ...
Please wait ...
Data downloaded to /Users/natebender/Desktop/repo/mslacounty_hvi/outputs/canopy_export_30063000300.tif
Exported GEOID: 30063000300
Gene